In [35]:
import os

In [9]:
%pwd

'/Users/ujjwaltiwari/projects/Kidney-Disease-Classifier/research'

In [10]:
os.chdir('../')

In [11]:
a = '🙀'

In [12]:
a

'🙀'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_model_path: Path
    training_data: Path
    params_epoch : int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [14]:
from DiseaseClassifier.constants import *
from DiseaseClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [19]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone")
        print(training_data)
        create_directories([
            Path(training.root_dir)
        ])
        print(training_data)

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epoch= params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size= params.IMAGE_SIZE,
            params_is_augmentation=params.AUGMENTATION
        )

        return training_config

In [20]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [21]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_model_path
        )

    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split =  0.25
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            class_mode="categorical",
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.25,
                height_shift_range=0.15,
                shear_range=0.17,
                zoom_range=0.19,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='training',
            shuffle=True,
            class_mode="categorical",
            **dataflow_kwargs
        )
        

    @staticmethod   
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_pre_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epoch,
            steps_per_epoch = self.steps_pre_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
        

In [22]:
try:
    early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)
    callback_list = [early_stopping]
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
except Exception as e: 
    raise e

[2024-04-10 14:27:57,596: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-10 14:27:57,599: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-10 14:27:57,599: INFO: common: Created directory at : artifacts]
artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone
[2024-04-10 14:27:57,600: INFO: common: Created directory at : artifacts/training]
artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone
[2024-04-10 14:28:01,346: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.]
Found 3110 images belonging to 4 classes.
Found 9336 images belonging to 4 classes.
Epoch 1/10


KeyboardInterrupt: 